# Optimize the models' hyperparameter

In this notebook, there is only one goal: optimize the hyperparameters of the models. 

In [4]:
# import all of the packages that will be used

# basic packages for data processing
import pandas as pd
import numpy as np

# packages for machine learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn import tree 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# packages to interpret the training result
from sklearn import metrics
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

# set the random seed to ensure the result is reproducible
import random
random.seed(10)
np.random.seed(10)

## Load the dataset

In [7]:
features = pd.read_csv('Dataset/features_selected/CIC-IDS2017_RandomForestClassifier_20.csv').squeeze()
features = features[:11]

In [9]:
columns = features.tolist() +  ['Label']

cic_ids2017 = pd.read_csv('Dataset/dataset_cleaned/CIC-IDS2017.csv', usecols=columns)
cic_ids2017.shape

(649762, 12)

## Processing before training

### Down sampling the dataset

As grid search is basically brute force all combinations of hyperparameters that are defined in the hyperparameter space, it is an expensive process. To speed up the process, only 10% of the training dataset will be used. 

In [13]:
cic_ids2017 = cic_ids2017.sample(frac=0.1).reset_index(drop=True)

print('Class distribution: ')
cic_ids2017['Label'].value_counts()

Class distribution: 


Label
malicious    32520
benign       32456
Name: count, dtype: int64

### Split the dataset

Split the features and the output variable into two dataframe, and further split them into testing and training dataset. The ratio between the training and testing dataset is 8:2 (i.e. 80% for training and 20% for testing). The training dataset will be used to optimize the models' hyperparameter while the testing dataset will give us a first glance on the accuracy of the optimized models. 

In [16]:
ids2017_X = cic_ids2017.drop('Label', axis=1).copy()
ids2017_y = cic_ids2017['Label'].copy()

ids2017_train_X, ids2017_test_X, ids2017_train_y, ids2017_test_y = train_test_split(ids2017_X, ids2017_y, test_size=0.3)

### Data scaling

In [19]:
scalar = StandardScaler()

ids2017_train_X_scaled = scalar.fit_transform(ids2017_train_X)
ids2017_test_X_scaled = scalar.transform(ids2017_test_X)

## Optimize Hyperparameter of each models

### Decision Tree

The decision tree will be optimized by tuning the `ccp_alpha` value. A for loop is being used to loop through multiple `ccp_alpha` values and the alpha value that gives the highest accuracy score will be selected. 

In [23]:
%%time

decision_tree = tree.DecisionTreeClassifier(criterion='entropy')
path = decision_tree.cost_complexity_pruning_path(ids2017_train_X_scaled, ids2017_train_y)
ccp_alphas = path.ccp_alphas
ccp_alphas = ccp_alphas[:-1] # exlude the maximum value

alpha_loop_values = []

for ccp_alpha in ccp_alphas:
    decision_tree = tree.DecisionTreeClassifier(criterion='entropy', ccp_alpha=ccp_alpha)
    scores = cross_val_score(decision_tree, ids2017_train_X_scaled, ids2017_train_y, cv=5)
    alpha_loop_values.append([ccp_alpha, np.mean(scores), np.std(scores)])


# Plot a graph of the means and standard deviations of the scores for each candidate
alpha_results = pd.DataFrame(alpha_loop_values, columns=['alpha', 'mean_accuracy', 'std'])

alpha_results.plot(x='alpha', 
                    y='mean_accuracy', 
                    yerr='std', 
                    marker='o', 
                    linestyle='--')

CPU times: total: 1min 32s
Wall time: 1min 32s


<Axes: xlabel='alpha'>

Build the optimal tree and evaluate its accuracy

In [26]:
ideal_ccp_alpha = alpha_results.sort_values(by=['mean_accuracy'], ascending=False).iloc[0]
ideal_ccp_alpha = float(ideal_ccp_alpha['alpha'])
print(f"Optimal alpha value: {ideal_ccp_alpha}")

# build the optimum tree
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', ccp_alpha=ideal_ccp_alpha)
decision_tree.fit(ids2017_train_X_scaled, ids2017_train_y)

prediction = decision_tree.predict(ids2017_test_X_scaled)
accuracy = metrics.accuracy_score(ids2017_test_y, prediction)
print(f"Accuracy: {accuracy:.5f}")

Optimal alpha value: 1.4401469385343852e-05
Accuracy: 0.99646


## Random Forest

In [29]:
%%time

parameter_space = [
    {'criterion': ['gini', 'entropy'],
    'max_depth': [10, 20, 30],
    'min_samples_leaf': [3, 0.0001, 0.0005, 0.00001], 
    'min_samples_split': [8, 0.0005, 0.0001, 0.00001], 
    'n_estimators': [100, 200, 350]
    }
]

optimal_rf = GridSearchCV(
                        RandomForestClassifier(),
                        parameter_space, 
                        cv=5,
                        scoring='accuracy',
                        n_jobs=-1,
                        verbose=0
)

optimal_rf.fit(ids2017_train_X_scaled, ids2017_train_y)
rf_optimal_params = optimal_rf.best_params_
print(f"Optimum hyperparameters: \n{rf_optimal_params}")

Optimum hyperparameters: 
{'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1e-05, 'min_samples_split': 1e-05, 'n_estimators': 200}
CPU times: total: 15.3 s
Wall time: 20min 25s


In [31]:
prediction = optimal_rf.predict(ids2017_test_X_scaled)
accuracy = metrics.accuracy_score(ids2017_test_y, prediction)
print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.99672


## Support Vector Machine

In [34]:
%%time
param_grid = [
    {'C': [1, 10, 100],
    'gamma': ['scale', 1, 0.1, 'auto'],
    'kernel': ['rbf', 'linear', 'sigmoid']}
]

optimal_svm = GridSearchCV(
                        SVC(),
                        param_grid, 
                        cv=5,
                        scoring='accuracy',
                        n_jobs=-1,
                        verbose=0
)

optimal_svm.fit(ids2017_train_X_scaled, ids2017_train_y)
svm_optimal_params = optimal_svm.best_params_
print(f"Optimum hyperparameters: \n{svm_optimal_params}")

Optimum hyperparameters: 
{'C': 100, 'gamma': 1, 'kernel': 'rbf'}
CPU times: total: 29 s
Wall time: 24min 36s


Make prediction using the optimum model and evaluate its accuracy

In [37]:
prediction = optimal_svm.predict(ids2017_test_X_scaled)
accuracy = metrics.accuracy_score(ids2017_test_y, prediction)
print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.95793


## Naive Bayes

In [40]:
%%time

parameter_space = [
    {'var_smoothing': np.logspace(0, -9, num=100)}
]

optimal_nb = GridSearchCV(
                        GaussianNB(),
                        parameter_space, 
                        cv=5,
                        n_jobs=-1,
                        verbose=0
)

optimal_nb.fit(ids2017_train_X_scaled, ids2017_train_y)
nb_optimal_params = optimal_nb.best_params_
print(f"Optimum hyperparameters: \n{nb_optimal_params}")

Optimum hyperparameters: 
{'var_smoothing': 1.0}
CPU times: total: 1.72 s
Wall time: 5.27 s


Make prediction using the optimum model and evaluate its accuracy

In [43]:
prediction = optimal_nb.predict(ids2017_test_X_scaled)
accuracy = metrics.accuracy_score(ids2017_test_y, prediction)
print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.73570


## Artificial Neural Network

In [46]:
%%time

parameter_space = [
    {'hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.00001]}
]

ann = MLPClassifier(max_iter=500)

optimal_ann = GridSearchCV(
                        ann,
                        parameter_space, 
                        cv=5,
                        n_jobs=-1,
                        verbose=0
)

optimal_ann.fit(ids2017_train_X_scaled, ids2017_train_y)
ann_optimal_params = optimal_ann.best_params_
print(f"Optimum hyperparameters: \n{ann_optimal_params}")

Optimum hyperparameters: 
{'activation': 'tanh', 'alpha': 1e-05, 'hidden_layer_sizes': (40,), 'solver': 'adam'}
CPU times: total: 3min 31s
Wall time: 29min 47s


In [48]:
prediction = optimal_ann.predict(ids2017_test_X_scaled)
accuracy = metrics.accuracy_score(ids2017_test_y, prediction)
print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.95239


## Deep Neural Network

In [51]:
%%time

parameter_space = [
    {'hidden_layer_sizes': [(10,10, 10), (12, 12, 12), (15, 15, 15), (8, 8, 8, 8), (10, 10, 10, 10)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.00001, 0.001]}
]

dnn = MLPClassifier(max_iter=500)

optimal_dnn = GridSearchCV(
                        dnn,
                        parameter_space, 
                        cv=5,
                        n_jobs=-1,
                        verbose=0
)

optimal_dnn.fit(ids2017_train_X_scaled, ids2017_train_y)
dnn_optimal_params = optimal_dnn.best_params_
print(f"Optimum hyperparameters: \n{dnn_optimal_params}")

Optimum hyperparameters: 
{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (15, 15, 15), 'solver': 'adam'}
CPU times: total: 42.1 s
Wall time: 30min 33s


In [53]:
prediction = optimal_dnn.predict(ids2017_test_X_scaled)
accuracy = metrics.accuracy_score(ids2017_test_y, prediction)
print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.97466
